#SQLLite RAG

This example shows how to use regular text to query weather data in the SQLLite database.



In [40]:
# pip install -U langchain langchain-community langchain-ollama

import re
from langchain_ollama import ChatOllama
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_core.prompts import ChatPromptTemplate

# 1) Build DB with sample rows included in table_info
db = SQLDatabase.from_uri(
    "sqlite:///weather.db",
    sample_rows_in_table_info=3,          # include 3 sample rows per table
    indexes_in_table_info=False,          # optional
    max_string_length=200                 # keep samples compact
)

# 2) Local model
llm = ChatOllama(model="llama3.1", temperature=0)

# 3) Prompt that shows schema + samples and enforces SQL-only
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a SQLite expert. Given an input question, create a syntactically correct {dialect} query to run.\n"
     "Unless otherwise specified, do not return more than {top_k} rows.\n"
     "Use only the following tables, columns, and sample rows:\n{table_info}\n\n"
     "Return only a single SQL statement. No explanations, no markdown, no code fences."),
    ("human", "{input}")
])

# 4) Chain with required variables (create_sql_query_chain injects table_info/top_k)
chain = create_sql_query_chain(llm, db, prompt=prompt)

# 5) Defensive extractor to strip any stray prose/markdown
SQL_PATTERN = re.compile(r'(?is)\\b(SELECT|WITH|INSERT|UPDATE|DELETE)\\b.*?;', re.DOTALL)
def extract_sql(text: str) -> str:
    cleaned = text.replace("``````", "").strip()
    m = SQL_PATTERN.search(cleaned.replace("\n", " "))
    return m.group(0).strip() if m else cleaned

# 6) Ask a question and run
resp = chain.invoke({"question": "What was the hottest day in each month for each year?"})
sql = extract_sql(resp)
#print("SQL:", sql)

rsp = db.run(sql)
print(sql,"\n")
rsp = rsp.split(",")
 

print("\n".join(str(item) for item in rsp))


SELECT datetime_utc, MAX(temp) FROM weather_obs GROUP BY strftime('%Y-%m', datetime_utc) 

[('2024-01-21T12:00:00'
 25.4)
 ('2024-02-13T12:00:00'
 21.6)
 ('2024-03-31T15:00:00'
 26.2)
 ('2024-04-24T17:00:00'
 33.3)
 ('2024-05-10T11:00:00'
 29.6)
 ('2024-06-07T15:00:00'
 37.6)
 ('2024-07-28T16:00:00'
 34.6)
 ('2024-08-17T13:00:00'
 33.1)
 ('2024-09-07T14:00:00'
 32.1)
 ('2024-10-02T14:00:00'
 28.8)
 ('2024-11-01T11:00:00'
 27.0)
 ('2024-12-06T15:00:00'
 22.6)
 ('2025-01-14T13:00:00'
 21.0)
 ('2025-02-01T14:00:00'
 19.1)
 ('2025-03-28T10:00:00'
 29.2)
 ('2025-04-01T12:00:00'
 25.4)
 ('2025-05-11T14:00:00'
 30.1)
 ('2025-06-29T14:00:00'
 31.9)
 ('2025-07-27T16:00:00'
 33.8)
 ('2025-08-14T18:00:00'
 35.8)
 ('2025-09-12T15:00:00'
 32.0)]


In [21]:
# Show the exact schema+samples string that will go into the prompt
table_info = db.get_table_info()
print(table_info)  # DDL plus sample rows per table (because of sample_rows_in_table_info) [web:463]

# Or fetch a few rows manually from a table
print(db.run("SELECT * FROM weather_obs LIMIT 3;"))  # returns a tabular string [web:465]



CREATE TABLE weather_obs (
	id INTEGER, 
	name TEXT, 
	lat REAL, 
	lon REAL, 
	datetime_utc TEXT NOT NULL, 
	"temp" REAL, 
	feelslike REAL, 
	dew REAL, 
	humidity REAL, 
	precip REAL, 
	precipprob REAL, 
	preciptype TEXT, 
	snow REAL, 
	snowdepth REAL, 
	windgust REAL, 
	windspeed REAL, 
	winddir REAL, 
	sealevelpressure REAL, 
	cloudcover REAL, 
	visibility REAL, 
	solarradiation REAL, 
	solarenergy REAL, 
	uvindex REAL, 
	severerisk REAL, 
	conditions TEXT, 
	icon TEXT, 
	stations_raw TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from weather_obs table:
id	name	lat	lon	datetime_utc	temp	feelslike	dew	humidity	precip	precipprob	preciptype	snow	snowdepth	windgust	windspeed	winddir	sealevelpressure	cloudcover	visibility	solarradiation	solarenergy	uvindex	severerisk	conditions	icon	stations_raw
1	34.68,32.61	34.68	32.61	2024-07-25T00:00:00	27.8	32.9	25.8	88.66	0.0	0.0	None	0.0	0.0	26.6	9.6	326.0	1001.3	85.6	24.6	0.0	0.0	0.0	None	Partially cloudy	partly-cloudy-night	LCRA,17601099999,F1785,LCPH,1